# Comparing image classification models side by side

In this tutorial, we will download two models from the [ELL gallery](https://microsoft.github.io/ELL/gallery/) and run them side-by-side on a Raspberry Pi. Some of the models on the gallery are slower abd accurate, while others are faster but less accurate. Running two models as once allows us to get a sense of their relative speeds and accuracies. Specifically, we will compare a standard (real valued) Convolutional Neural Network to a Neural Network that contains binarized layers. The binarized model is smaller and faster, but less accurate. 

![screenshot](https://microsoft.github.io/ELL/tutorials/Comparing-Image-Classification-models-side-by-side-on-the-Raspberry-Pi/Screenshot.jpg)


We start by installing the `ell` package.

In [ ]:
!conda config --prepend channels conda-forge --prepend channels microsoft-ell
!conda install -y ell

Start by loading and building two models, one a real-valued ELL model, one a binarized ELL model.

In [ ]:
from ell.pretrained_model import PretrainedModel
import ell.platform

pretrained1 = PretrainedModel('d_I160x160x3CMCMCMCMCMCMC1A')
pretrained1.download('comparing', rename='model1')
pretrained1.build()

In [ ]:
pretrained2 = PretrainedModel('d_I160x160x3NCMNCMNBMNBMNBMNBMNC1A')
pretrained2.download('comparing', rename='model2')
categories = [line.strip('\n') for line in open('comparing/categories.txt', 'r').readlines()]
pretrained2.build()

Next, load both models.

In [ ]:
model1 = pretrained1.load()
model2 = pretrained2.load()
predictions = model1.FloatVector(model.get_default_output_shape().Size())

Both models take the same size input, so prepare a test image to pass to the `predict` method of each model.

In [ ]:
import urllib.request
testurl = "https://upload.wikimedia.org/wikipedia/commons/7/73/07-08_Chevrolet_Suburban_LT.jpg"

import urllib.request
import cv2
import numpy as np
urllib.request.urlretrieve(testurl, 'comparing/testimage.jpg')
image = cv2.imread('comparing/testimage.jpg')
from IPython.display import Image
display(Image('comparing/testimage.jpg'))

input = pretrained_model.prepare_image(image)

Run the `predict` to use each model to predict the image category. Use the %time magic to report how long it takes to run the cell code. 

In [ ]:
%time
model1.predict(input, predictions)
top5 = helpers.get_top_n_predictions(predictions, 5)
print("".join(["(" + str(int(conf * 100)) + "%) " + categories[index] for (index, conf) in top5]))

Repeat the same step for the second model.

In [ ]:
%time
model2.predict(input, predictions)
top5 = helpers.get_top_n_predictions(predictions, 5)
print("".join(["(" + str(int(conf * 100)) + "%) " + categories[index] for (index, conf) in top5]))

We would expect the second (binarized) model to be faster but less accurate than the first (real-valued) model.